In [1]:
# pip install langdetect
# pip install sentencepiece
# pip install boto3
# pip install awscli
# pip install sacremoses

In [2]:
import gradio as gr
from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM
from dotenv import load_dotenv
import os
import subprocess
import torch
import tempfile
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer
import re
import boto3

In [3]:
# import functions from functions file

from functions_mm import handle_query, transcribe_audio_original, submit_question,polly_text_to_speech, translate, translate_and_speech, clear_inputs, voice_map, language_map, default_language, languages



In [8]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
instructions = """
# Diabetes Chatbot
### Step 1: Record your audio OR input text (NOT both!)
### Step 2: Would you like response in new language? Choose your language  
### Step 3: Submit question
### Step 4: Translate response
### Step 5: Clear inputs and start fresh
"""


with gr.Blocks() as app2:
    
    with gr.Row():
        gr.Markdown(instructions)
        
    with gr.Row():
        input_audio = gr.Audio(
            label="Ask a question about Diabetes, then click 'Transcribe audio",
            type="filepath")
        language_dropdown = gr.Dropdown(label="Click the middle of the dropdown bar to select translation language",
                                        choices=list(language_map.keys()), value=default_language, type='value')
        
    with gr.Row():
        transcribe_button = gr.Button("Transcribe audio")
        submit_button = gr.Button("Submit your question")
        translate_button = gr.Button("Translate the response")
        clear_button = gr.Button("Clear All")
    
    #Divide the screen horizontally into 2 columns
    with gr.Row():
            #This column will be on the left side of screen
            with gr.Column():
                query_text = gr.Textbox(label="Type your question here. If there is audio recorded AND question text, app will submit question text. Click transcribe button to populate with audio text")
                # output_original_speech = gr.Audio(label="Text to speech here")
            
            with gr.Column():
                response_text = gr.Textbox(label="Chatbot response")
                response_speech = gr.Audio(label="Chatbot response speech")    
        
            #This column will be on the right side of screen
            with gr.Column():    
                output_translated = gr.Textbox(label="Translated text")
                output_translated_speech = gr.Audio(label="Translated speech")

    # Audio transcription
    transcribe_button.click(
        fn=transcribe_audio_original,
        inputs=[input_audio],
        outputs=[query_text]
    )
    
    submit_button.click(
        fn=submit_question,
        inputs=[input_audio, query_text, language_dropdown],
        outputs=[response_text, response_speech]
    )
        
    # Translation
    translate_button.click(
        fn=translate_and_speech,
        inputs=[response_text, language_dropdown],
        outputs=[output_translated, output_translated_speech]
    )
        
    #Clearing all inputs and outputs
    clear_button.click(
    fn=clear_inputs,
    inputs=[],
    outputs=[input_audio, query_text, response_text, response_speech, output_translated, output_translated_speech]
    )

app2.launch(show_error=True, share=True)






Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://558fcb0f6bd798ee24.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\mered\anaconda3\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\

An error occurred: Helsinki-NLP/opus-mt-es-trk is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
An error occurred: Helsinki-NLP/opus-mt-es-hi is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
An error occurred: Helsinki-NLP/opus-mt-es-jap is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
An error occurred: Helsinki-NLP/opu

Traceback (most recent call last):
  File "c:\Users\mered\anaconda3\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\gradio\blocks.py", line 1786, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\gradio\blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^